In [ ]:
import sys
import os


sys.path.append(os.path.abspath("../..")) #make the folder "automl" part of this

RESULTS_PATH = 'results.csv'
OPTUNA_DATABASE = 'study_results.db'
BASE_CONFIGURATION_NAME = 'configuration'


In [ ]:
from automl.loggers.result_logger import ResultLogger
import optuna
import optuna.visualization as vis
from automl.utils.optuna_utils import load_study_from_database
import matplotlib.pyplot as plt


# Load the experiment

In [ ]:
if False:

    experiments_base_path = "C:\\Users\\rgoncalo\\Universidade\\ricardo-goncalo-thesis-project\\project\\examples\\simple_metarl\\archived_data\\sb3_montaincar_semi_trained_reduced_or_not"
    experiment_specific_path = "sb3_montaincar_semi_trained_200"
    
    experiment_path = f"{experiments_base_path}\\{experiment_specific_path}\\Experiment"
    #experiment_path = 'data\\experiments\\HyperparameterOptimizationPipeline_0'
    #experiment_path = 'archived_data\\first'

In [ ]:
if False:

    #hp_model_name = 'sb3_CartPole_dqn_gaussian_0_0.05_0.3'
    #hp_model_name = 'sb3_CartPole_dqn_gaussian_0_0.2_0.8'
    #hp_model_name = 'sb3_CartPole_dqn'
    #hp_model_name = 'sb3_CartPole_dqn_perturbed_0_10'
    hp_model_name = 'sb3_CartPole_dqn_forgotten_100_1e-17'
    
    
    hp_experiment_name=f'sb3_zoo_dqn_cartpole_hp_opt'
    
    
    experiment_path = f'C:\\rgoncalo\\experiments\\{hp_experiment_name}\\experiments\\{hp_model_name}'

In [ ]:
if True:

    #base_experiment_path = "C:\\Experiments\\rl-zoo-CartPole-dqn\\HPOpt\\1\\experiments"
    base_experiment_path = "C:\\Experiments\\rl-zoo-CartPole-dqn\\HPOpt\\4\\experiments"

    #hp_model_name = 'sb3_CartPole_dqn_gaussian_0_0.05_0.3'
    #hp_model_name = 'sb3_CartPole_dqn_gaussian_0_0.2_0.8'
    #hp_model_name = 'sb3_CartPole_dqn'
    hp_model_name = 'sb3_CartPole_dqn_perturbed_0_10'
    #hp_model_name = 'sb3_CartPole_dqn_forgotten_100_1e-17'
    
    experiment_path = f'{base_experiment_path}\\{hp_model_name}'

In [ ]:
if not os.path.exists(experiment_path):
    raise Exception("DOES NOT EXIST")

# Evaluation of HyperparameterOptimizationPipeline

In [ ]:
from automl.meta_rl.hp_eval_results.hp_eval_results import get_hp_opt_results_logger

hyperparameter_optimization_results = get_hp_opt_results_logger(experiment_path)

print(f"Hyperparameter_optimization_results in path: {hyperparameter_optimization_results.get_artifact_directory()}")

In [ ]:
from automl.meta_rl.hp_eval_results.hp_eval_results import get_hp_opt_optuna_study


optuna_study = get_hp_opt_optuna_study(hyperparameter_optimization_results)


## Hyperparameter Study

In [ ]:
try:
    print(f"optuna_study done with with best value {optuna_study.best_value} in trial {optuna_study.best_trial.number} with best parameters:\n{optuna_study.best_params}")

except:
    print("No best trial yet")

In [ ]:
from automl.meta_rl.hp_eval_results.hp_eval_results import print_optuna_trials_info

print_optuna_trials_info(optuna_study)

In [ ]:
from optuna.visualization import plot_slice

fig = plot_slice(optuna_study)
fig.show()

In [ ]:
from automl.meta_rl.hp_eval_results.hp_eval_results import print_optuna_param_importances

print_optuna_param_importances(optuna_study)

In [ ]:
fig = vis.plot_param_importances(optuna_study)
fig.show()

In [ ]:


fig = vis.plot_parallel_coordinate(optuna_study)
fig.show()

In [ ]:
fig = vis.plot_intermediate_values(optuna_study)
fig.show()

In [ ]:
fig = vis.plot_optimization_history(optuna_study)
fig.show()

In [ ]:
from automl.meta_rl.hp_eval_results.hp_eval_results import plot_scattered_values_for_param

try:
    plot_scattered_values_for_param(optuna_study)

except Exception as e:
    print(f"Could not plot scattered values because of error: {e}")

In [ ]:
parameters_to_plot = []
# parameters_to_plot = ["hidden_size", "hidden_layers"]

if len(parameters_to_plot) > 1:

    fig = vis.plot_contour(optuna_study, params=parameters_to_plot)
    fig.show()

# Global evaluation of configurations

In [ ]:
AGGREGATE_NUMBER = 10 #the number of neighbor points to sum to plot the needed graphs

In [ ]:
from automl.meta_rl.hp_eval_results.hp_eval_results import study_of_configuration                          


In [ ]:
from automl.meta_rl.hp_eval_results.hp_eval_results import get_results_of_configurations                          



results_of_configurations : dict[str, ResultLogger] = get_results_of_configurations(experiment_path)

In [ ]:
print(f"Configurations:  {results_of_configurations.keys()}")

# Global view of performance

In [ ]:
hyperparameter_optimization_results.plot_bar_graph(x_axis='experiment', y_axis='result')

## Pruned Performances

In [ ]:
from automl.meta_rl.hp_eval_results.hp_eval_results import get_pruned_trials                          


pruned_optuna_trials, pruned_optuna_trials_per_steps, pruned_trials = get_pruned_trials(optuna_study)

print(f"Pruned trials: {pruned_trials}")


### Worst Pruned Trials

In [ ]:
WORST_PRUNED = 3

In [ ]:
for n_completed_steps in pruned_optuna_trials_per_steps.keys():
    
    pruned_optuna_trials : list[optuna.Trial] = pruned_optuna_trials_per_steps[n_completed_steps][0:WORST_PRUNED] 
    
    pruned_trials = [f'configuration_{trial.number}' for trial in pruned_optuna_trials ]

    for configuration_name in pruned_trials:
        
        results_logger = results_of_configurations[configuration_name]
    
        study_of_configuration(configuration_name, results_logger)

### Best Pruned Trials

In [ ]:
BEST_PRUNED = 3

In [ ]:
for n_completed_steps in pruned_optuna_trials_per_steps.keys():
    
    #correct this
    pruned_optuna_trials : list[optuna.Trial] = pruned_optuna_trials_per_steps[n_completed_steps][-BEST_PRUNED:] 
    
    pruned_trials = [f'configuration_{trial.number}' for trial in pruned_optuna_trials ]

    for configuration_name in pruned_trials:
        
        results_logger = results_of_configurations[configuration_name]
    
        study_of_configuration(configuration_name, results_logger)

## Completed Trials Evaluation

In [ ]:
completed_optuna_trials = [trial for trial in optuna_study.trials if trial.state == optuna.trial.TrialState.COMPLETE]

completed_optuna_trials.sort(key=lambda trial: trial.value) # sort given the trial value


### Worst Completed Performances

In [ ]:
N_WORST = 3

In [ ]:

worst_optuna_trials = completed_optuna_trials[0:N_WORST]

worst_configurations_to_study = [f"configuration_{trial.number}" for trial in worst_optuna_trials]

print(f"Worst configurations to study: {worst_configurations_to_study}")

In [ ]:
for configuration_name in worst_configurations_to_study:
    
    results_logger = results_of_configurations[configuration_name]

    study_of_configuration(configuration_name, results_logger)



## Non Completed Performances

In [ ]:
non_completed_optuna_trials = [trial for trial in optuna_study.trials if trial.state == optuna.trial.TrialState.RUNNING or trial.state == optuna.trial.TrialState.WAITING]

non_completed_optuna_trials.sort(key=lambda trial: trial.value) # sort given the trial value

### All non completed trials

In [ ]:
non_completed_trials = [f"configuration_{trial.number}" for trial in non_completed_optuna_trials]


In [ ]:
for configuration_name in non_completed_trials:
    
    results_logger = results_of_configurations[configuration_name]

    study_of_configuration(configuration_name, results_logger)

## Best Performances

In [ ]:
N_BEST = 3

In [ ]:
best_optuna_trials = completed_optuna_trials[(len(completed_optuna_trials) - N_BEST):]

best_configurations_to_study = [f"configuration_{trial.number}" for trial in best_optuna_trials]

print(f"Best configurations to study: {best_configurations_to_study}")

In [ ]:
for configuration_name in best_configurations_to_study:
    
    results_logger = results_of_configurations[configuration_name]

    study_of_configuration(configuration_name, results_logger)

# Smaller study

In [ ]:
configurations_to_study = ['configuration_0']

In [ ]:

try:

    for configuration_name in configurations_to_study:
    
        results_logger = results_of_configurations[configuration_name]

        study_of_configuration(configuration_name, results_logger)

except KeyError as e:
    print(f"KeyError: {e}\nAvailable keys are {results_of_configurations.keys()}")
    raise e

In [ ]:
agents_in_study = []
# agents_in_study = ["agent_1", "agent_2"]

In [ ]:
agents_to_study : dict[str, ResultLogger]= {}

for configuration_name in configurations_to_study:
    
    results_logger = results_of_configurations[configuration_name]
    
    for agent_name in agents_in_study:
      
        agent_results_logger = ResultLogger(input={
                                            "logger_directory" : f"{results_logger.lg.logDir}\\{agent_name}",
                                            "filename" : RESULTS_PATH,
                                            "create_new_directory" : False
                                          })

        agents_to_study[f"{configuration_name}_{agent_name}"] = agent_results_logger
        
        agent_results_logger.proccess_input()


In [ ]:
for agent_name, agent_results_logger in agents_to_study.items():
    
    study_of_configuration(agent_name, agent_results_logger)